In [1]:
%pip install torch
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install coremltools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder

In [32]:
# Set up the paths
HOME_PATH = os.path.expanduser('~')
MODELS_PATH = f'{HOME_PATH}/Developer/BU/research/models'
DATASET_PATH = f'../../../data/'
data_features = f'{DATASET_PATH}/WISDM_x.csv'
data_labels = f'{DATASET_PATH}/WISDM_y.csv'

In [30]:
# Load Data
x = genfromtxt(data_features, delimiter=',')
y_df = pd.read_csv(data_labels)
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [4]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)
print(f'X_train shape:{X_train.shape}, X_test shape:{X_test.shape}, y_train shape:{y_train.shape}, y_test shape:{y_test.shape}')

X_train shape:(104856, 16, 3), X_test shape:(26214, 16, 3), y_train shape:(104856,), y_test shape:(26214,)


In [5]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_steps):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=n_input, out_channels=256, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 8, 6)  # Adjusted for the output from the last Conv1d layer
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.dropout(torch.relu(self.conv1(x)))
        x = self.dropout(torch.relu(self.conv2(x)))
        x = self.dropout(torch.relu(self.conv3(x)))
        x = self.dropout(torch.relu(self.conv4(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return self.softmax(x)

# Example instantiation of the model
model = MyModel(n_input=3, n_steps=16)


In [6]:
# Convert arrays to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # Assuming y_train is class labels for classification
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Creating TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Creating DataLoaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [7]:
import torch.optim as optim

# Setting up the device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)  # Reshape input to match model expectations
        #print(f'Input size after reshaping: {inputs.size()}')

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.view(-1, 3, 16)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    # Print training/validation statistics
    print(f'Epoch: {epoch+1}/{epochs} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f} \tValidation Accuracy: {val_accuracy:.2f}%')

print('Training complete')


Epoch: 1/20 	Training Loss: 1.3195 	Validation Loss: 1.2905 	Validation Accuracy: 74.99%
Epoch: 2/20 	Training Loss: 1.2648 	Validation Loss: 1.2421 	Validation Accuracy: 80.00%
Epoch: 3/20 	Training Loss: 1.2471 	Validation Loss: 1.2332 	Validation Accuracy: 80.88%
Epoch: 4/20 	Training Loss: 1.2369 	Validation Loss: 1.2394 	Validation Accuracy: 80.20%
Epoch: 5/20 	Training Loss: 1.2216 	Validation Loss: 1.1798 	Validation Accuracy: 86.27%
Epoch: 6/20 	Training Loss: 1.1983 	Validation Loss: 1.2006 	Validation Accuracy: 84.23%
Epoch: 7/20 	Training Loss: 1.1952 	Validation Loss: 1.1855 	Validation Accuracy: 85.76%
Epoch: 8/20 	Training Loss: 1.1960 	Validation Loss: 1.1847 	Validation Accuracy: 85.84%
Epoch: 9/20 	Training Loss: 1.1967 	Validation Loss: 1.1787 	Validation Accuracy: 86.45%
Epoch: 10/20 	Training Loss: 1.2013 	Validation Loss: 1.2049 	Validation Accuracy: 83.83%
Epoch: 11/20 	Training Loss: 1.2089 	Validation Loss: 1.1824 	Validation Accuracy: 86.10%
Epoch: 12/20 	Train

In [8]:
# Test phase - after training is complete
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 3, 16)
        print(f'Input size after reshaping: {inputs.size()}')
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

# Print test statistics
print(f'Test Loss: {test_loss:.4f} \tTest Accuracy: {test_accuracy:.2f}%')


Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size after reshaping: torch.Size([64, 3, 16])
Input size a

In [9]:
model_path = f'{MODELS_PATH}/CNN_base.pth'
torch.save(model.state_dict(), model_path)
model.to(device)

MyModel(
  (conv1): Conv1d(3, 256, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
  (conv3): Conv1d(128, 64, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=256, out_features=6, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=1)
)

### Coreml Evaluation

In [10]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for i in range(len(X_test)):
        X_test_sample = X_test[i].view(1, 3, 16)
        #X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_sample.numpy()})
        pred_class = np.argmax(output_dict['var_75'])
        predictions.append(pred_class)
    
    accuracy = np.sum(np.array(predictions) == y_test.numpy()) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

### Convert to coreml model

In [34]:
# Load the model
model_path = f'{MODELS_PATH}/CNN_base.pth'
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [35]:
import coremltools as ct
example_input = torch.rand(1,3,16) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
coreml_model.save(f'{MODELS_PATH}/cnn.mlpackage')

Running MIL default pipeline:   0%|          | 0/71 [00:00<?, ? passes/s]/opt/homebrew/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:267: UserWarning: Output, '75', of the source model, has been renamed to 'var_75' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 8053.06 passes/s]


In [36]:
coreml_model

input {
  name: "x"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 16
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_75"
  type {
    multiArrayType {
      shape: 1
      shape: 6
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}

In [37]:
import coremltools as ct
import coremltools.optimize.coreml as cto

cnn_coreml_model = ct.models.MLModel(f'{MODELS_PATH}/cnn.mlpackage')
cnn_coreml_model

input {
  name: "x"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 16
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_75"
  type {
    multiArrayType {
      shape: 1
      shape: 6
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}

In [38]:
model_name = cnn_coreml_model
model_path = f'{MODELS_PATH}/cnn.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.8402380407415885
Size of the model: 0.12 KB


## Post Training Optimization

In [55]:
from coremltools.optimize.coreml import (
    OpThresholdPrunerConfig,
    OpMagnitudePrunerConfig,
    OpPalettizerConfig,
    OpLinearQuantizerConfig,
    OptimizationConfig,
    prune_weights,
)

### Quantization

In [39]:
op_config = OpLinearQuantizerConfig(
    mode="linear_symmetric", weight_threshold=512
)
config = OptimizationConfig(global_config=op_config)

compressed_8_bit_model = cto.linear_quantize_weights(cnn_coreml_model, config=config)

Running compression pass linear_quantize_weights: 100%|██████████| 9/9 [00:00<00:00, 1637.19 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 5610.48 passes/s]


In [40]:
compressed_8_bit_model.save(f'{MODELS_PATH}/cnn_8bitQuantized_mlmodel.mlpackage')

In [41]:
model_name = compressed_8_bit_model
model_path = f'{MODELS_PATH}/cnn_8bitQuantized_mlmodel.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.8402761882963302
Size of the model: 0.12 KB


### Pruning

#### a) OpMagnitudePrunerConfig: Prune the weights with a constant sparsity percentile

In [45]:
op_config = OpMagnitudePrunerConfig(
    target_sparsity=0.6,
    weight_threshold=1024,
)
config = OptimizationConfig(global_config=op_config)
#magnitude_pruner = prune_weights(mlmodel, config=config)
cnn_magnitude_pruner = prune_weights(cnn_coreml_model, config=config)


Running compression pass prune_weights: 100%|██████████| 9/9 [00:00<00:00, 731.71 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 5014.11 passes/s]


In [46]:
cnn_magnitude_pruner.save(f'{MODELS_PATH}/cnn_magnitude_pruner.mlpackage')

In [47]:
model_name = cnn_magnitude_pruner
model_path = f'{MODELS_PATH}/cnn_magnitude_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.6924544136720836
Size of the model: 0.12 KB


#### b) OpThresholdPrunerConfig: Sets all weight values below a certain value.

In [50]:
op_config = OpThresholdPrunerConfig(
    threshold=0.001,
    minimum_sparsity_percentile=0.01,
    weight_threshold=1024,
)

config = OptimizationConfig(global_config=op_config)
#magnitude_pruner = prune_weights(mlmodel, config=config)
cnn_threshold_pruner = prune_weights(cnn_coreml_model, config=config)

Running compression pass prune_weights:   0%|          | 0/9 [00:00<?, ? ops/s]weight value has sparsity of 0.0013020833333333333 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.007527669270833333 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0079345703125 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.00390625 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.005208333333333333 < minimum_sparsity_percentile 0.01. Skipped.
Running compression pass prune_weights: 100%|██████████| 9/9 [00:00<00:00, 2748.16 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 7532.42 passes/s]


In [51]:
cnn_threshold_pruner.save(f'{MODELS_PATH}/cnn_threshold_pruner.mlpackage')

In [52]:
model_name = cnn_threshold_pruner
model_path = f'{MODELS_PATH}/cnn_threshold_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.8402380407415885
Size of the model: 0.12 KB


### Palletization

In [56]:
op_config = OpPalettizerConfig(
    mode="kmeans", 
    nbits=6
)

config = OptimizationConfig(global_config=op_config)
cnn_palettizer = cto.palettize_weights(cnn_coreml_model, config=config)

Running compression pass palettize_weights: 100%|██████████| 9/9 [00:00<00:00, 11.33 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 4259.62 passes/s]


In [57]:
cnn_palettizer.save(f'{MODELS_PATH}/cnn_palettizer.mlpackage')

In [58]:
model_name = cnn_palettizer
model_path = f'{MODELS_PATH}/cnn_palettizer.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.8400473029678798
Size of the model: 0.12 KB
